# Calculate histogram of each input image

## Requirements
- A folder with images 

## Config

### The following code imports and declares functions used for the processing:

In [ ]:
#################################
#  Don't modify the code below  #
#################################

import os
import re
import time
import numpy as np
import intake_io
import pandas as pd
from tqdm import tqdm
from am_utils.utils import walk_dir

## Specify data paths and analysis parameters

### Please provide data paths:

`input_dir`: folder with images of cells to be processed

`output_dir`: folder to save results

In [ ]:
input_dir = "/research/sharedresources/cbi/common/Anna/test/input"
output_dir = "/research/sharedresources/cbi/common/Anna/test/histogram"

### The following code lists all image files in the input directory:

In [ ]:
#################################
#  Don't modify the code below  #
#################################

samples = walk_dir(input_dir)

print(f'{len(samples)} images were found:')
print(np.array(samples))

### The following code computes histograms for all images

In [ ]:
#################################
#  Don't modify the code below  #
#################################

all_hist = pd.DataFrame()

for sample in tqdm(samples):
    dataset = intake_io.imload(sample)

    imghist = pd.DataFrame()
    for i in range(dataset.dims['c']):
        img = dataset.loc[dict(c=dataset.coords['c'][i])]['image'].data
        hist, bins = np.histogram(img, bins=np.max(img)+1, range=(0, np.max(img)+1))
        chist = pd.DataFrame({
            'values': bins[:-1],
            'counts': hist
        })
        chist = chist[chist['counts'] > 0]
        chist['channel'] = dataset.coords['c'][i].data
        imghist = pd.concat([imghist, chist], ignore_index=True)
    imghist['Image name'] = sample
    fn_out = sample.replace(input_dir, output_dir).replace('.'+sample.split('.')[-1], '.csv')
    os.makedirs(os.path.dirname(fn_out), exist_ok=True)
    imghist.to_csv(fn_out, index=False)
    all_hist = pd.concat([all_hist, imghist], ignore_index=True)
all_hist.to_csv(output_dir + '.csv', index=False)